LOAD THƯ VIỆN

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from concurrent.futures import ThreadPoolExecutor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer,IterativeImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import minimize
import xgboost as xgb
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go
from IPython.display import display

HÀM TIỆN ÍCH

In [ ]:
# Map giá trị season thành số
def map_season(df):
    season_map = {'Missing': 0, 'Spring': 1, 'Summer': 2, 'Fall': 3, 'Winter': 4}
    for column in df.columns:
        if 'Season' in column:
            df[column] = df[column].map(season_map)
    return df

# Điền missing values bằng KNN Imputer
def imputeByKNN(df):
    imputer = KNNImputer(n_neighbors=5)
    df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    return df_imputed

# Load dữ liệu timeseries và tính thống kê
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"Stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    
    return df

# Tối ưu ngưỡng dự đoán
def map_to_sii(pciat_total, thresholds):
    if pciat_total <= thresholds[0]:
        return 0
    elif pciat_total <= thresholds[1]:
        return 1
    elif pciat_total <= thresholds[2]:
        return 2
    else:
        return 3

def threshold_optimizer(thresholds, y_true, oof_non_rounded):
    rounded_predictions = np.array([map_to_sii(pred, thresholds) for pred in oof_non_rounded])
    return -cohen_kappa_score(y_true, rounded_predictions, weights='quadratic')

# Vẽ biểu đồ về số lượng ô trống trung bình (phần trăm) của từng cột
def plot_missing_values(df):
    missing_counts = df.isnull().mean() * 100
    missing_counts = missing_counts[missing_counts > 0].sort_values(ascending=False)

    plt.figure(figsize=(30, 18))
    sns.barplot(x=missing_counts.index, y=missing_counts.values)
    plt.xticks(rotation=45, ha='right')
    plt.title('Phần trăm ô trống trung bình trong từng cột')
    plt.ylabel('Phần trăm ô trống (%)')
    plt.xlabel('Tên cột')
    plt.tight_layout()
    plt.show()

# Hiển thị bảng thể hiện tương quan với cột "PCIAT-PCIAT_Total"
def display_correlation_with_target(df, target_column):
    correlation_matrix = df.corr()
    target_correlation = correlation_matrix[target_column].sort_values(ascending=False)
    correlation_df = target_correlation.reset_index()
    correlation_df.columns = ['Column', 'Correlation']
    print(correlation_df)
    return correlation_df

# Hàm đổi các cột trong bảng thành numeric
def convert_to_numeric(df):
    for col in df.columns:
        if df[col].dtype == 'object' or df[col].dtype.name == 'category':
            df[col], _ = pd.factorize(df[col])
    return df

LOAD DỮ LIỆU

In [ ]:
input_dir = "/kaggle/input/child-mind-institute-problematic-internet-use"
print("Dữ liệu có trong thư mục:", os.listdir(input_dir))

# Load dữ liệu tabular và timeseries
print("Loading tabular data...")
tabular_train_df = pd.read_csv(input_dir + "/train.csv")
tabular_test_df_with_id = pd.read_csv(input_dir + "/test.csv")
data_dict = pd.read_csv(input_dir + "/data_dictionary.csv")

print("Loading timeseries data...")
timeseri_train = load_time_series(input_dir + "/series_train.parquet")
timeseri_test = load_time_series(input_dir + "/series_test.parquet")

# Merge tabular với timeseries
tabular_train_df = pd.merge(tabular_train_df, timeseri_train, how="left", on='id')
tabular_test_df_with_id = pd.merge(tabular_test_df_with_id, timeseri_test, how="left", on='id')

# Xóa cột id
tabular_train_df = tabular_train_df.drop(columns='id', axis=1)
tabular_test_df = tabular_test_df_with_id.drop(columns='id', axis=1)

BIỂU ĐỒ

In [ ]:
# Phần trăm các ô trống trong từng cột
plot_missing_values(tabular_train_df)

In [ ]:
train_df = tabular_train_df.copy()
train_df = convert_to_numeric(train_df)

In [ ]:
# tương quan giữa các cột của bảng train và cột PCIAT-PCIAT_Total
display_correlation_with_target(train_df, target_column='PCIAT-PCIAT_Total')

In [ ]:
# tương quan giữa các cột của bảng train và cột sii
display_correlation_with_target(train_df, target_column='sii')

In [ ]:
# bảng train
display(tabular_train_df.head(10).style.set_table_styles(
    [{'selector': 'thead th', 'props': [('font-size', '12pt'), ('text-align', 'center')]},
     {'selector': 'tbody td', 'props': [('font-size', '10pt'), ('text-align', 'center')]}]
))

PRETRAIN DATA

In [ ]:
# Lấy các cột chung giữa tập train và test
common_columns = list(set(tabular_train_df.columns) & set(tabular_test_df.columns))

# Giữ lại các cột chung và thêm các cột cần thiết trong tập train
train_columns = common_columns + ['sii', 'PCIAT-PCIAT_Total']
tabular_train_df = tabular_train_df[train_columns]

# Chỉ giữ lại các cột chung trong tập test
tabular_test_df = tabular_test_df[common_columns]

# Loại bỏ các hàng có giá trị NaN ở cột "sii"
tabular_train_df = tabular_train_df.dropna(subset=['sii'])

# Điền missing values cho dữ liệu "Season"
season_columns = [col for col in tabular_train_df.columns if 'Season' in col]
tabular_train_df[season_columns] = tabular_train_df[season_columns].fillna('Missing')
tabular_test_df[season_columns] = tabular_test_df[season_columns].fillna('Missing')
tabular_train_df = map_season(tabular_train_df)
tabular_test_df = map_season(tabular_test_df)

# Điền missing values bằng KNN Imputer
tabular_train_df = imputeByKNN(tabular_train_df)
tabular_test_df = imputeByKNN(tabular_test_df)
#tabular_train_df = imputeByIterative(tabular_train_df)
#tabular_test_df = imputeByIterative(tabular_test_df)

# Tách X (features) và y (target)
X_train = tabular_train_df.drop(['sii', 'PCIAT-PCIAT_Total'], axis=1)
y_train = tabular_train_df['PCIAT-PCIAT_Total']

X_test = tabular_test_df

TRAIN MODEL

In [ ]:
xgb_params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,
    'reg_lambda': 5,
    'random_state': 42,
    'tree_method': 'exact'
}

def train_model():
    initial_thresholds = [30, 49, 79]  # Các ngưỡng khởi tạo
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Định nghĩa các mô hình thành phần
    model_xgb = XGBRegressor(
        learning_rate=0.05, max_depth=6, n_estimators=200, subsample=0.8,
        colsample_bytree=0.8, reg_alpha=1, reg_lambda=5, random_state=42, tree_method='exact')

    model_lgbm = LGBMRegressor(
        learning_rate=0.05, max_depth=6, n_estimators=200, subsample=0.8,
        colsample_bytree=0.8, reg_alpha=1, reg_lambda=5, random_state=42)

    model_rf = RandomForestRegressor(
        n_estimators=200, max_depth=6, random_state=42)

    # Voting ensemble kết hợp các mô hình trên
    voting_model = VotingRegressor(
        estimators=[('xgb', model_xgb), ('lgbm', model_lgbm), ('rf', model_rf)])

    oof_non_rounded = np.zeros(len(y_train), dtype=float)
    test_preds = np.zeros((len(X_test), 5))  # Dự đoán cho test qua từng fold

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        voting_model.fit(X_fold_train, y_fold_train)

        y_val_predicts = voting_model.predict(X_fold_val)
        oof_non_rounded[val_idx] = y_val_predicts
        test_preds[:, fold] = voting_model.predict(X_test)

        fold_kappa = cohen_kappa_score(y_fold_val, np.round(y_val_predicts), weights='quadratic')
        print(f"Fold {fold + 1} - QWK: {fold_kappa:.4f}")

    # Tối ưu hóa thresholds để đạt QWK cao nhất
    kappa_optimizer = minimize(threshold_optimizer, x0=initial_thresholds, args=(y_train, oof_non_rounded), method='Nelder-Mead')
    assert kappa_optimizer.success, "Threshold optimization failed"

    optimized_thresholds = kappa_optimizer.x
    print(f"Optimized thresholds: {optimized_thresholds}")

    # Dự đoán trên tập test
    y_pred = test_preds.mean(axis=1)
    y_pred_sii = [map_to_sii(pred, optimized_thresholds) for pred in y_pred]

    # Tạo submission
    submission = pd.DataFrame({
        'id': tabular_test_df_with_id['id'],
        'sii': y_pred_sii
    })
    submission.to_csv('submission.csv', index=False)

    print("\nTraining and testing complete. Submission saved as 'submission.csv'.")

RUN PREDICTION

In [ ]:
train_model()